In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [178]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [64]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

In [78]:
size = 128

In [79]:
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model2.fit(x_train, y_train, epochs=100, batch_size=1024)

In [39]:
model2.fit(x_train, y_train, epochs=5, batch_size=1024)

Epoch 1/5
59/59 [==============================] - 0s 3ms/step - loss: 0.5487 - accuracy: 0.8127
Epoch 2/5
59/59 [==============================] - 0s 3ms/step - loss: 0.4680 - accuracy: 0.8416
Epoch 3/5
59/59 [==============================] - 0s 3ms/step - loss: 0.4360 - accuracy: 0.8500
Epoch 4/5
59/59 [==============================] - 0s 3ms/step - loss: 0.4119 - accuracy: 0.8576
Epoch 5/5
59/59 [==============================] - 0s 3ms/step - loss: 0.3904 - accuracy: 0.8639


In [7]:
model2.fit(x_train, y_train, epochs=5, batch_size=256)

Epoch 1/5
235/235 [==============================] - 1s 2ms/step - loss: 0.1609 - accuracy: 0.9402
Epoch 2/5
235/235 [==============================] - 1s 3ms/step - loss: 0.1578 - accuracy: 0.9408
Epoch 3/5
235/235 [==============================] - 1s 2ms/step - loss: 0.1541 - accuracy: 0.9426
Epoch 4/5
235/235 [==============================] - 1s 2ms/step - loss: 0.1514 - accuracy: 0.9437
Epoch 5/5
235/235 [==============================] - 1s 3ms/step - loss: 0.1476 - accuracy: 0.9455


In [8]:
model2.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2197 - accuracy: 0.9174
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1923 - accuracy: 0.9264
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1866 - accuracy: 0.9294
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1770 - accuracy: 0.9331
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1736 - accuracy: 0.9339


In [80]:
tester_model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
tester_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [81]:
model2.evaluate(x_test, y_test, verbose=2, batch_size=256)

40/40 - 0s - loss: 2.6039 - accuracy: 0.0998


[2.6038713455200195, 0.0997999981045723]

In [83]:
l, a = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
for i in range(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
    elif change > 0:
        num_worse += 1
        wr += [i]
    else:
        num_important += 1
        imp += [i]

40/40 - 0s - loss: 2.6039 - accuracy: 0.0998
Node 0: 0.00010000169277191162
Node 1: 0.0
Node 2: -0.00010000169277191162
Node 3: 0.00010000169277191162
Node 4: 0.00020000338554382324
Node 5: -0.00019999593496322632
Node 6: 0.0
Node 7: 0.00010000169277191162
Node 8: 0.0
Node 9: 0.0
Node 10: -0.00010000169277191162
Node 11: 0.00010000169277191162
Node 12: 0.0
Node 13: -0.00010000169277191162
Node 14: -0.00019999593496322632
Node 15: -0.00010000169277191162
Node 16: 0.0
Node 17: 0.0
Node 18: -0.0005999952554702759
Node 19: -0.00010000169277191162
Node 20: 0.0
Node 21: -0.00029999762773513794
Node 22: -0.00010000169277191162
Node 23: -0.0018999949097633362
Node 24: -0.00029999762773513794
Node 25: -0.0014999955892562866
Node 26: 0.0
Node 27: 0.00020000338554382324
Node 28: 0.0
Node 29: 0.0
Node 30: -0.00010000169277191162
Node 31: -0.00010000169277191162
Node 32: -0.00019999593496322632
Node 33: 0.0
Node 34: 0.00010000169277191162
Node 35: -0.0006999969482421875
Node 36: 0.0
Node 37: 0.0
No

In [84]:
print("Zero Nodes:", num_zeros)
print("Worse Nodes:", num_worse)
print("Important Nodes:", num_important)

Zero Nodes: 12
Worse Nodes: 69
Important Nodes: 47


In [48]:
print("######## IMPORTANT NODES ########")
for i in imp:
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.*(na - a) + 1.0*(l - nl))

######## IMPORTANT NODES ########
Node 0: -0.011357992887496948
Node 1: -0.0003668367862701416
Node 2: -0.01113802194595337
Node 3: -0.004285454750061035
Node 4: -0.006938248872756958
Node 7: -0.009644269943237305
Node 8: -0.004353582859039307
Node 9: -0.011253029108047485
Node 10: -0.005652725696563721
Node 11: -0.005932420492172241
Node 12: -0.012002736330032349
Node 13: -0.003979533910751343
Node 14: -0.002023845911026001
Node 18: -0.0007498264312744141
Node 20: -0.013696044683456421
Node 21: -0.003553926944732666
Node 22: -0.02637985348701477
Node 23: -0.0009359419345855713
Node 26: -0.0028370916843414307
Node 27: -0.0482616126537323
Node 28: -0.007632553577423096
Node 29: -0.005993574857711792
Node 32: -0.00876551866531372
Node 33: -0.009699732065200806
Node 34: -0.002858489751815796
Node 35: -0.03807932138442993
Node 36: -0.007287859916687012
Node 37: -0.005283772945404053
Node 38: -0.029007047414779663
Node 39: -0.022837698459625244
Node 40: -0.006233334541320801
Node 43: -0.004

In [49]:
print("######## WORSE NODES ########")
tot = 0
for i in wr:
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.*(na - a) + 1.0*(l - nl))
    tot += (l - nl)
print(tot)
print(tot / num_worse)

######## WORSE NODES ########
Node 5: 0.006364375352859497
Node 6: 0.005842745304107666
Node 15: 0.0006026327610015869
Node 16: 0.0008873343467712402
Node 17: 0.0024060606956481934
Node 19: 0.0041099488735198975
Node 25: 0.000403672456741333
Node 30: 0.003470808267593384
Node 51: 0.0010623335838317871
Node 55: 0.003407120704650879
Node 58: 0.0009611845016479492
Node 64: 0.0014757513999938965
Node 65: 0.00016021728515625
Node 68: 5.054473876953125e-05
Node 79: 0.0002141892910003662
Node 83: 0.008299171924591064
Node 84: 0.0002543032169342041
Node 86: 0.0018314719200134277
Node 91: 0.00015357136726379395
Node 94: 0.0015983879566192627
Node 102: 0.005549639463424683
Node 105: 0.005634099245071411
Node 106: 0.0014474093914031982
Node 112: 0.0030728578567504883
Node 117: 0.0007586181163787842
Node 120: 0.000581204891204834
Node 121: 0.0006135106086730957
Node 125: 0.0022307932376861572
Node 126: 0.000684201717376709
Node 127: 0.000967562198638916
0.06509572267532349
0.0021698574225107827


In [86]:
loss, acc = model2.evaluate(x_train, y_train, verbose=2, batch_size=512)
end_not_reached = True
improved = False
tol = -1e-5
current_pos = 0
best_pos = -1
best_change = tol
original2 = model2.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        #tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_train, y_train, verbose=0, batch_size=1024)
    if 0.3*(na - oa) + 0.7*(ol - nl) > best_change:
        best_change = 0.3*(na - oa) + 0.7*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%200 == 0:
        print("Did 200 iterations")

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

98/98 - 0s - loss: 2.6008 - accuracy: 0.0989
0.006856966018676757
Found something better
0.015709051489830015
Found something better
0.030262802541255948
Found something better
0.044894503057003016
Found something better
0.046278446912765496
Found something better
Improvement has occured!! Accuracy: 0.09638000279664993 --- Loss: 2.5336530208587646 --- Change: 0.046278446912765496 --- New tol: -1e-05
0.005933973938226699
Found something better
0.011274542659521103
Found something better
0.020122868567705152
Found something better
0.03969955369830132
Found something better
Improvement has occured!! Accuracy: 0.09625999629497528 --- Loss: 2.4768879413604736 --- Change: 0.03969955369830132 --- New tol: -1e-05
0.004554710537195206
Found something better
0.009817975014448165
Found something better
0.017311021685600277
Found something better
0.022616217285394667
Found something better
0.0349423386156559
Found something better
Improvement has occured!! Accuracy: 0.10294000059366226 --- Loss: 2

MemoryError: Unable to allocate 586. MiB for an array with shape (50000, 32, 32, 3) and data type float32

In [ ]:
num_removed2

In [ ]:
y_pred = model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())
AUC_or = m.result().numpy()
acc_or = acc

In [ ]:
y_pred = tester_model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())
AUC_red = m.result().numpy()
acc_red = acc

In [ ]:
print((np.array(acc_red) - np.array(acc_or)))
print(AUC_red - AUC_or)

In [ ]:
new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - num_removed)), np.zeros((best_weights[1].shape[0] - num_removed)), np.zeros((best_weights[2].shape[0] - num_removed, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1

In [ ]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128-num_removed, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.set_weights(new_weights)
red_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
import tqdm

In [18]:
l, a = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)

y_pred = model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
best_weights2 = model2.get_weights()

for i in tqdm.trange(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    # print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    elif change > 0:
        num_worse += 1
        wr += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    else:
        num_important += 1
        imp += [i]

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

y_pred = tester_model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

40/40 - 0s - loss: 0.3335 - accuracy: 0.8889


  0%|                                                                                         | 0/1024 [00:00<?, ?it/s]

[0.855, 0.968, 0.861, 0.927, 0.738, 0.973, 0.677, 0.952, 0.969, 0.969]
tf.Tensor(0.9906246, shape=(), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████████| 1024/1024 [01:31<00:00, 11.21it/s]


313/313 - 0s - loss: 0.5710 - accuracy: 0.7640
[0.549, 0.99, 0.041, 0.639, 0.9, 0.943, 0.84, 0.974, 0.974, 0.79]
tf.Tensor(0.980825, shape=(), dtype=float32)


In [24]:
l, a = model2.evaluate(x_train, y_train, verbose=2, batch_size=256)

y_pred = model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
best_weights2 = model2.get_weights()

for i in tqdm.trange(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_train, y_train, verbose=0, batch_size=x_train.shape[0])
    # print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    elif change > 0:
        num_worse += 1
        wr += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    else:
        num_important += 1
        imp += [i]

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

y_pred = tester_model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

235/235 - 0s - loss: 0.2042 - accuracy: 0.9226


  0%|                                                                                         | 0/1024 [00:00<?, ?it/s]

[0.855, 0.968, 0.861, 0.927, 0.738, 0.973, 0.677, 0.952, 0.969, 0.969]
tf.Tensor(0.9906246, shape=(), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████████| 1024/1024 [03:58<00:00,  4.29it/s]


313/313 - 0s - loss: 0.4497 - accuracy: 0.8302
[0.645, 0.985, 0.572, 0.682, 0.857, 0.893, 0.846, 0.984, 0.971, 0.867]
tf.Tensor(0.9865412, shape=(), dtype=float32)


In [23]:
print(num_zeros, num_worse, num_important)

0 1024 0


In [25]:
l, a = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)

y_pred = model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

or_weights = model2.get_weights()
tol_low = 0
tol_high = 0
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
best_weights2 = model2.get_weights()

for i in tqdm.trange(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    # print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    elif change > 0:
        num_worse += 1
        wr += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    else:
        num_important += 1
        imp += [i]

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

y_pred = tester_model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

40/40 - 0s - loss: 0.3335 - accuracy: 0.8889


  0%|▏                                                                                | 2/1024 [00:00<01:29, 11.40it/s]

[0.855, 0.968, 0.861, 0.927, 0.738, 0.973, 0.677, 0.952, 0.969, 0.969]
tf.Tensor(0.9906246, shape=(), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████████| 1024/1024 [01:28<00:00, 11.58it/s]


313/313 - 0s - loss: 0.5746 - accuracy: 0.7623
[0.534, 0.991, 0.045, 0.623, 0.885, 0.95, 0.858, 0.972, 0.979, 0.786]
tf.Tensor(0.98067045, shape=(), dtype=float32)


In [26]:
l, a = model2.evaluate(x_train, y_train, verbose=2, batch_size=256)

y_pred = model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
best_weights2 = model2.get_weights()

for i in tqdm.trange(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_train, y_train, verbose=0, batch_size=x_train.shape[0])
    # print(f"Node {i}:", 1.*(na - a) + 0*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    elif change > 0:
        num_worse += 1
        wr += [i]
        best_weights2[0][:,i] = 0
        best_weights2[1][i] = 0
        best_weights2[2][i,:] = 0
    else:
        num_important += 1
        imp += [i]

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

y_pred = tester_model2.predict(x_test)

K = len(set(y_test))
yp = tf.argmax(y_pred, axis=1)
acc = []
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]).numpy())
    acc.append(a)
print(acc)

m = tf.keras.metrics.AUC()
m.update_state(tf.one_hot(y_test, 10), y_pred)
print(m.result())

235/235 - 0s - loss: 0.2042 - accuracy: 0.9226


  0%|                                                                                         | 0/1024 [00:00<?, ?it/s]

[0.855, 0.968, 0.861, 0.927, 0.738, 0.973, 0.677, 0.952, 0.969, 0.969]
tf.Tensor(0.9906246, shape=(), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████████| 1024/1024 [03:51<00:00,  4.43it/s]


313/313 - 0s - loss: 0.4497 - accuracy: 0.8302
[0.645, 0.985, 0.572, 0.682, 0.857, 0.893, 0.846, 0.984, 0.971, 0.867]
tf.Tensor(0.9865412, shape=(), dtype=float32)


In [27]:
m.interpolate_pr_auc()

<tf.Tensor: shape=(), dtype=float32, numpy=0.9207167>

In [28]:
for i in range(len(set(y_test))):
    print(len(y_test[y_test==i]))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [270]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

In [271]:
size = 128

In [282]:
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
base_weights = model2.get_weights()
#model2.fit(x_train, y_train, epochs=100, batch_size=1024)

In [285]:
tester_model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
tester_model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [286]:
model2.evaluate(x_test, y_test, verbose=2, batch_size=256)

40/40 - 0s - loss: 2.4628 - accuracy: 0.0822 - auc_35: 0.4969


[2.462815523147583, 0.08219999819993973, 0.4969491958618164]

In [305]:
l, a, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
for i in range(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    # print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
    elif change > 0:
        num_worse += 1
        wr += [i]
    else:
        num_important += 1
        imp += [i]

40/40 - 0s - loss: 2.2859 - accuracy: 0.1606 - auc_35: 0.5724


In [306]:
print("Zero Nodes:", num_zeros)
print("Worse Nodes:", num_worse)
print("Important Nodes:", num_important)

Zero Nodes: 48
Worse Nodes: 5
Important Nodes: 75


In [304]:
print("######## WORSE NODES ########")
limit1 = np.sqrt(6 / (3200))
limit2 = np.sqrt(6 / (138))
w = copy.deepcopy(or_weights)
for i in wr:
    w[0][:,i] = list(np.random.uniform(-limit, limit, 3072))
    w[1][i] = 0
    w[2][i,:] = list(np.random.uniform(-limit, limit, 10))
model2.set_weights(w)

######## WORSE NODES ########


In [307]:
model2.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8554 - accuracy: 0.3289 - auc_35: 0.7858
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6917 - accuracy: 0.3952 - auc_35: 0.8293
Epoch 3/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6301 - accuracy: 0.4200 - auc_35: 0.8431
Epoch 4/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5900 - accuracy: 0.4339 - auc_35: 0.8516
Epoch 5/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5651 - accuracy: 0.4434 - auc_35: 0.8566
Epoch 6/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5436 - accuracy: 0.4531 - auc_35: 0.8610
Epoch 7/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5251 - accuracy: 0.4583 - auc_35: 0.8646
Epoch 8/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5075 - accuracy: 0.4657 - auc_35: 0.8680
Epoch 9/20
1563/1563 [==========================

In [308]:
model2.evaluate(x_test, y_test, verbose=2, batch_size=512)

20/20 - 0s - loss: 1.4929 - accuracy: 0.4737 - auc_35: 0.8716


[1.4928829669952393, 0.47369998693466187, 0.8715860247612]

In [309]:
blank_model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
blank_model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
blank_model2.set_weights(base_weights)
blank_model2.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8985 - accuracy: 0.3226 - auc_37: 0.7730
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7529 - accuracy: 0.3762 - auc_37: 0.8142
Epoch 3/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6944 - accuracy: 0.3975 - auc_37: 0.8281
Epoch 4/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.6618 - accuracy: 0.4074 - auc_37: 0.8358
Epoch 5/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.6320 - accuracy: 0.4193 - auc_37: 0.8424
Epoch 6/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.6089 - accuracy: 0.4270 - auc_37: 0.8474
Epoch 7/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.5892 - accuracy: 0.4332 - auc_37: 0.8516
Epoch 8/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5797 - accuracy: 0.4375 - auc_37: 0.8535
Epoch 9/20
1563/1563 [==========================

In [310]:
blank_model2.evaluate(x_test, y_test, verbose=2, batch_size=512)

20/20 - 0s - loss: 1.5919 - accuracy: 0.4362 - auc_37: 0.8523


[1.5918580293655396, 0.43619999289512634, 0.8522675037384033]

In [311]:
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model2.set_weights(base_weights)
#model2.fit(x_train, y_train, epochs=100, batch_size=1024)

In [312]:
tester_model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
tester_model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

In [313]:
model2.evaluate(x_test, y_test, verbose=2, batch_size=256)

40/40 - 0s - loss: 2.4628 - accuracy: 0.0822 - auc_38: 0.4969


[2.462815523147583, 0.08219999819993973, 0.4969491958618164]

In [330]:
l, a, auc = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
or_weights = model2.get_weights()
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
for i in range(size):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na, nauc = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    # print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
    elif change > 0:
        num_worse += 1
        wr += [i]
    else:
        num_important += 1
        imp += [i]

40/40 - 0s - loss: 2.9021 - accuracy: 0.1000 - auc_38: 0.4924


In [331]:
print("Zero Nodes:", num_zeros)
print("Worse Nodes:", num_worse)
print("Important Nodes:", num_important)

Zero Nodes: 22
Worse Nodes: 105
Important Nodes: 1


In [329]:
print("######## WORSE NODES ########")
limit1 = np.sqrt(6 / (3200))
limit2 = np.sqrt(6 / (138))
w = copy.deepcopy(or_weights)
for i in imp:
    w[0][:,i] = list(np.random.uniform(-limit, limit, 3072))
    w[1][i] = 0
    w[2][i,:] = list(np.random.uniform(-limit, limit, 10))
model2.set_weights(w)

######## WORSE NODES ########


In [332]:
model2.fit(x_train, y_train, epochs=20)

Epoch 1/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.9313 - accuracy: 0.2985 - auc_38: 0.7622
Epoch 2/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.8093 - accuracy: 0.3526 - auc_38: 0.7996
Epoch 3/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7746 - accuracy: 0.3642 - auc_38: 0.8083
Epoch 4/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7514 - accuracy: 0.3726 - auc_38: 0.8144
Epoch 5/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7362 - accuracy: 0.3758 - auc_38: 0.8182
Epoch 6/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7258 - accuracy: 0.3804 - auc_38: 0.8210
Epoch 7/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7183 - accuracy: 0.3819 - auc_38: 0.8229
Epoch 8/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7110 - accuracy: 0.3846 - auc_38: 0.8245
Epoch 9/20
1563/1563 [==========================

In [333]:
model2.evaluate(x_test, y_test, verbose=2, batch_size=512)

20/20 - 0s - loss: 1.7089 - accuracy: 0.3864 - auc_38: 0.8248


[1.7089338302612305, 0.3864000141620636, 0.8247711062431335]

In [188]:
l, a = model2.evaluate(x_test, y_test, verbose=2, batch_size=256)
tol_low = -1e-5
tol_high = 1e-5
num_zeros, num_worse, num_important = (0, 0, 0)
z = []
wr = []
imp = []
for i in tqdm.tqdm_notebook(range(size)):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=256)
    # print(f"Node {i}:", 0.*(na - a) + 1.*(l - nl))
    change = l - nl
    if change <= tol_high and change >= tol_low:
        num_zeros += 1
        z += [i]
    elif change > 0:
        num_worse += 1
        wr += [i]
    else:
        num_important += 1
        imp += [i]

40/40 - 0s - loss: 1.6823 - accuracy: 0.3956


C:\Users\lucas\Anaconda3\envs\master\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [189]:
print("Zero Nodes:", num_zeros)
print("Worse Nodes:", num_worse)
print("Important Nodes:", num_important)

Zero Nodes: 0
Worse Nodes: 0
Important Nodes: 64


In [169]:
print("######## WORSE NODES ########")
limit1 = np.sqrt(6 / (3200))
limit2 = np.sqrt(6 / (138))
w = copy.deepcopy(or_weights)
for i in wr:
    w[0][:,i] = list(np.random.uniform(-limit, limit, 3072))
    w[1][i] = 0
    w[2][i,:] = list(np.random.uniform(-limit, limit, 10))
model2.set_weights(w)

######## WORSE NODES ########


In [106]:
loss, acc = model2.evaluate(x_train, y_train, verbose=2, batch_size=512)
end_not_reached = True
improved = False
tol = -1e-5
current_pos = 0
best_pos = -1
best_change = tol
original2 = model2.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        #tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
    if 0.3*(na - oa) + 0.7*(ol - nl) > best_change:
        best_change = 0.3*(na - oa) + 0.7*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%200 == 0:
        print("Did 200 iterations")

tester_model2.set_weights(best_weights2)
loss2, acc2 = tester_model2.evaluate(x_test, y_test, verbose=2)

98/98 - 0s - loss: 2.3971 - accuracy: 0.0869
0.006078667193651199
Found something better
0.007046357542276381
Found something better
0.013568452745676038
Found something better
0.017282692342996595
Found something better
0.020366835594177242
Found something better
Improvement has occured!! Accuracy: 0.08150000125169754 --- Loss: 2.365630865097046 --- Change: 0.020366835594177242 --- New tol: -1e-05
0.005944054573774338
Found something better
0.009405882656574249
Found something better
0.011495045572519302
Found something better
Improvement has occured!! Accuracy: 0.09839999675750732 --- Loss: 2.356452226638794 --- Change: 0.011495045572519302 --- New tol: -1e-05
0.003387662023305893
Found something better
0.004780366271734238
Found something better
0.0072697028517723075
Found something better
Improvement has occured!! Accuracy: 0.10369999706745148 --- Loss: 2.3483383655548096 --- Change: 0.0072697028517723075 --- New tol: -1e-05
0.0024870097637176513
Found something better
0.0051490552

In [108]:
new_weights = [np.zeros((best_weights2[0].shape[0], best_weights2[0].shape[1] - num_removed2)),
               np.zeros((best_weights2[1].shape[0] - num_removed2)),
               np.zeros((best_weights2[2].shape[0] - num_removed2, best_weights2[2].shape[1])),
               best_weights2[3]]

j = 0
for i in range(len(best_weights2[1])):
    if i not in nodes_removed2:
        new_weights[0][:, j] = best_weights2[0][:, i]
        new_weights[1][j] = best_weights2[1][i]
        new_weights[2][j, :] = best_weights2[2][i, :]
        j = j + 1

In [122]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.set_weights(new_weights)
red_model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 2.2945 - accuracy: 0.1452


[2.2944507598876953, 0.1451999992132187]

In [135]:
num_removed2

0

In [141]:
red_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 2s 2ms/step - loss: 1.8282 - accuracy: 0.3363
Epoch 2/5
1563/1563 [==============================] - 2s 2ms/step - loss: 1.8217 - accuracy: 0.3417
Epoch 3/5
1563/1563 [==============================] - 2s 2ms/step - loss: 1.8176 - accuracy: 0.3409
Epoch 4/5
1563/1563 [==============================] - 2s 2ms/step - loss: 1.8143 - accuracy: 0.3409
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 1.8111 - accuracy: 0.3441


In [142]:
new_weights = red_model.get_weights()
model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.set_weights(new_weights)
model2.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 1.8012 - accuracy: 0.3435


[1.8012312650680542, 0.3434999883174896]

In [143]:
tester_model2 = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(size, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
tester_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [144]:
loss, acc = model2.evaluate(x_train, y_train, verbose=2, batch_size=512)
end_not_reached = True
improved = False
tol = -1e-5
current_pos = 0
best_pos = -1
best_change = tol
original2 = model2.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = model2.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        #tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    tester_model2.set_weights(w)
    nl, na = tester_model2.evaluate(x_test, y_test, verbose=0, batch_size=1024)
    if 0.*(na - oa) + 1.*(ol - nl) > best_change:
        best_change = 0.*(na - oa) + 1.*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%200 == 0:
        print("Did 200 iterations")

tester_model2.set_weights(best_weights2)

98/98 - 0s - loss: 1.7879 - accuracy: 0.3527
